In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

Using TensorFlow backend.


In [2]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

In [3]:
Y_train = train["label"]

# Drop 'label' column
X_train = train.drop(labels = ["label"],axis = 1) 

# free some space
del train 

In [4]:
# Normalize the data, change to -1,1
X_train = X_train / 127.5 - 1
test = test / 127.5 -1
# Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

In [5]:
Y_train = to_categorical(Y_train, num_classes = 10)

# set random seed for network, maybe not useful
from numpy.random import seed 
seed(42)

# Split the train and the validation set for the fitting
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.12, random_state=2)
print(X_train.shape, Y_train.shape)

(38220, 28, 28, 1) (38220, 10)


In [6]:
model = Sequential()

# conv1
model.add(Conv2D(filters = 96, kernel_size = (3,3),padding = 'same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 96, kernel_size = (3,3),padding = 'same', 
                 activation ='relu'))
model.add(Conv2D(filters = 96, kernel_size = (3,3),padding = 'same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

# conv3
model.add(Conv2D(filters = 96, kernel_size = (3,3),padding = 'same', 
                 activation ='relu'))
model.add(Conv2D(filters = 96, kernel_size = (3,3),padding = 'same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))


# flatten
model.add(Flatten())
model.add(Dense(1024, activation = "relu"))
model.add(Dropout(0.3))
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.3))
model.add(Dense(10, activation = "softmax"))
# Compile the model

opt = Adam(lr=0.001)
model.compile(optimizer = opt , loss = "categorical_crossentropy", metrics=["accuracy"])
print("compile done")

compile done


In [7]:
datagen = ImageDataGenerator(
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        ) 


datagen.fit(X_train)

/home/jac/.local/lib/python3.5/site-packages/keras/preprocessing/image.py:653: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (38220, 28, 28, 1) (1 channels).
  ' (' + str(x.shape[self.channel_axis]) + ' channels).')


In [8]:
from keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras.models import load_model

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=4, 
                                            verbose=1, 
                                            factor=0.6, 
                                            min_lr=0.00001)
    
best_model_path = 'wide_best_m_v2.h5'
best_model = ModelCheckpoint(filepath=best_model_path, monitor='val_loss', save_best_only=True, verbose=1)
epochs = 100
steps_per_epoch = 100
batch_size = 64
model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_val,Y_val),
                              verbose = 2, steps_per_epoch=steps_per_epoch,
                              callbacks=[best_model,learning_rate_reduction])

Epoch 1/60
Epoch 00000: val_loss improved from inf to 0.15086, saving model to wide_best_m_v2.h5
7s - loss: 0.9409 - acc: 0.6739 - val_loss: 0.1509 - val_acc: 0.9577
Epoch 2/60
Epoch 00001: val_loss improved from 0.15086 to 0.08461, saving model to wide_best_m_v2.h5
6s - loss: 0.2561 - acc: 0.9239 - val_loss: 0.0846 - val_acc: 0.9738
Epoch 3/60
Epoch 00002: val_loss improved from 0.08461 to 0.07541, saving model to wide_best_m_v2.h5
6s - loss: 0.1704 - acc: 0.9458 - val_loss: 0.0754 - val_acc: 0.9772
Epoch 4/60
Epoch 00003: val_loss improved from 0.07541 to 0.05996, saving model to wide_best_m_v2.h5
6s - loss: 0.1267 - acc: 0.9627 - val_loss: 0.0600 - val_acc: 0.9791
Epoch 5/60
Epoch 00004: val_loss did not improve
6s - loss: 0.1224 - acc: 0.9625 - val_loss: 0.0602 - val_acc: 0.9796
Epoch 6/60
Epoch 00005: val_loss improved from 0.05996 to 0.05210, saving model to wide_best_m_v2.h5
6s - loss: 0.1186 - acc: 0.9656 - val_loss: 0.0521 - val_acc: 0.9849
Epoch 7/60
Epoch 00006: val_loss imp

Epoch 00058: val_loss did not improve

Epoch 00058: reducing learning rate to 7.775999838486313e-05.
6s - loss: 0.0192 - acc: 0.9931 - val_loss: 0.0123 - val_acc: 0.9960
Epoch 60/60
Epoch 00059: val_loss improved from 0.01068 to 0.01040, saving model to wide_best_m_v2.h5
6s - loss: 0.0170 - acc: 0.9950 - val_loss: 0.0104 - val_acc: 0.9971


In [9]:
model = load_model(best_model_path)  # try callback model
# predict results
results = model.predict(test)
# select the indix with the maximum probability
results = np.argmax(results,axis = 1)
results = pd.Series(results,name="Label")
import arrow
ts = str(arrow.now())[:-10].replace(':','_').replace('.',"_")
res_file = "wide_play_cnn_mnist_datagen_{}.csv".format(ts)
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)
submission.to_csv(res_file,index=False)
print('save done',res_file)

save done wide_play_cnn_mnist_datagen_2017-09-22T12_03_50_85.csv
